# Install and import libraries

In [ ]:
!pip install lxml
!pip install selenium
!pip install cssselect
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 10.5MB/s 
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubun

In [ ]:
import pandas as pd
import numpy as np

from lxml import html
from lxml.cssselect import CSSSelector

# Create the web driver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


# Load the site

In [ ]:
url = "https://ph.investing.com/equities/philippines"
driver.get(url)
driver.implicitly_wait(10)

# Show all stocks
filter = driver.find_element_by_css_selector("#stocksFilter")
filter.find_element_by_css_selector("option#all").click()

# Create a list where we will store our links
stock_links = []

try:
  # This is the css selector for the anchor tag that contains our desired url
  links_selector = "#cross_rate_markets_stocks_1 tbody tr td:nth-child(2) a"
  # We will use WebDriverWait() to ensure that the element has actually loaded before we try and retrieve it
  anchors = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, links_selector)))

  # Once we get the list of anchor elements, we will iterate through each of them and store
  # the stock name and url to our stock_links list
  for anchor in anchors:
    anchor_href = anchor.get_attribute("href").split("?")
    stock_links.append({
        "name": anchor.get_attribute("title"),
        "url": anchor_href[0] + "-historical-data" + "?" + "?".join(anchor_href[1:])
    })

  # Show a confirmation that the process is successful and how many stocks did we collected
  print("Retrieved", len(stock_links), "URLs")
except TimeoutException:
  print("Element not properly loaded")

Retrieved 303 URLs


In [ ]:
def get_historical_data(stocks):
  # List where we will store our collected data
  historical_data = []
  # A variable to keep track of our progress
  count = 0

  # We'll iterate through all of our collected URLs
  for stock in stocks:
    count += 1
    try:
      # List where we will temporarily store the historical data for a specific stock code
      stock_data = []

      # Load the url on our web driver
      driver.get(stock["url"])
      driver.implicitly_wait(10)

      # We would have to modify the filters such that we can get the oldest
      # data as possible

      # This line selects the widget for the date and excecute a javascript code
      # to perform the click event for us
      date_widget = driver.find_element_by_id("widgetField")
      driver.execute_script("arguments[0].click();", date_widget)

      # We would now set the start date field to 08/08/1927, date where the PSE
      # was originally founded
      start_date = driver.find_element_by_id("startDate")
      start_date.clear()
      start_date.send_keys("08/08/1927")

      # Finally, we would apply our changes to update our displayed data
      apply_widget = driver.find_element_by_id("applyBtn")
      driver.execute_script("arguments[0].click();", apply_widget)

      # Now we would get the only h2 element in the page which contains the stock code
      stock_code = driver.find_element_by_tag_name("h2").get_attribute("innerHTML")
      stock_code = stock_code.split(" ")[0]

      # We would now extract the table body's HTML source from the page
      # Note: The table body contains the all the stock price data that we need
      table_body = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#curr_table tbody")))

      # We would convert the HTML source that we get into a form that LXML works on
      table_html = table_body[0].get_attribute('innerHTML')
      table_tree = html.fromstring(table_html)

      # Using LXML, we would isolate all table row into a list
      rows = table_tree.cssselect("tr")

      for row in rows:
        # From this table row, we would extract all the content of every table cell
        # and store it to our list
        temp = []
        for col in row.cssselect("td"):
          temp.append(col.text)

        # To make our dataset easier to navigate, we will also add the stock
        # name and stock code to our list
        temp = [stock["name"], stock_code] + temp
        stock_data.append(temp)

      # We would now add this newly collected data to our historical_data list
      historical_data += stock_data

      # To keep track and now that our script is working, we would print a message
      # containing a summary of the scraping's progress
      print("(", count, "/ 303 ) Retrieved", stock_code, "with", len(stock_data), "rows [", stock_data[0][2], " - ", stock_data[-1][2], "]")
    except Exception as e:
      # We might also encounter an error where our collection script fails,
      # in the rare case where this would happen we would need to manually
      # revisit what went wrong and fix it.
      print("(", count, "/ 303 ) Error occured while retrieving", stock["name"])

  return historical_data

In [ ]:
# Declare our column names
columns = ["Stock Name", "Code", "Date", "Price", "Open", "High", "Low", "Volume", "Change%"]

# Invoke the function that scrapes the data
all_data = get_historical_data(stock_links)

# Create a dataframe containing our data and declared columns
pd_data = pd.DataFrame(all_data, columns=columns)

# Finally, let us export it to a CSV file or any file format that you want
pd_data.to_csv("stocks.csv", index=False)